## 업종분류 현황 및 개별종목 지표 크롤링
주식 관련 데이터를 구하기 위해 가장 먼저 해야 하는 일은 어떠한 종목들이 상장되어 있는가에 대한 정보를 구하는 것이다.  
한국거래소에서 제공하는 업종분류 현황과 개별종목 지표 데이터를 이용하면 매우 간단하게 해당 정보를 수집할 수 있다.

#### 업종분류 현황 크롤링

In [1]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'   # 1
data = rq.get(url)   # 2
data_html = BeautifulSoup(data.content)   # 3
parse_day = data_html.select_one(   # 4
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah'   # 4
).text

print(parse_day)

  |  2025.03.12


1. 페이지 주소를 입력한다.
2. get() 함수를 통해 해당 페이지 내용을 받아 온다.
3. BeautifulSoup() 함수를 이용해 해당 페이지의 HTML 내용을 BeautifulSoup 객체로 만든다.
4. select_one() 메서드를 통해 해당 태그의 데이터를 추출하며, text 메서드를 이용해 텍스트 데이터만을 추출한다.

In [2]:
import re

biz_day = re.findall('[0-9]+', parse_day)   # 1
biz_day = ''.join(biz_day)   # 2

print(biz_day)

20250312


1. findall() 메서드 내에 정규 표현식을 이용해 숫자에 해당하는 부분만을 추출한다.  
   '[0-9]+'는 모든 숫자를 의미하는 표현식이다.
2. join() 함수를 통해 숫자를 합쳐 준다.

In [3]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'   # 1
gen_otp_stk = {
    'mktId': 'STK',   # 2
    'trdDd': biz_day,   # 3
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader',   # 4
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text   # 5

print(otp_stk)

O9MzsznmDY+tK2myNPAO+ckn98zKYV9KI6Nh8zbDXgsRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eQIQos+z+exSmF+62+Df+WwtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


1. get_otp_url에 원하는 항목을 제출할 URL을 입력한다.
2. 개발자 도구 화면에 있는 쿼리 내용들을 딕셔너리 형태로 입력한다. 이 중 mktId의 'STK'는 코스피에 해당하며,
   코스닥 데이터를 받고자 할 경우 'KSQ'를 입력하면 된다.
3. 영업일을 뜻하는 trdDd에는 위에서 구한 최근 영업일 데이터를 입력한다.
4. 헤더 부분에 리퍼러(Referer)를 추가한다. 리퍼러란 링크를 통해서 각각의 웹사이트로 방문할 때 남는 흔적이다.
   거래서 데이터를 다운로드하는 과정을 살펴보면 첫 번째 URL에서 OTP를 부여받고, 이를 다시 두 번째 URL에 제출했다.
   그런데 이러한 과정의 흔적이 없이 OTP를 바로 두 번째 URL에 제출하면 서버는 이를 로봇으로 인식해 데이터를 주지 않는다.
   이러한 리퍼러 주소는 개발자 도구 화면에서도 확인할 수 있다.
5. post() 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이 중 텍스트에 해당하는 내용만 불러온다.

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'   # 1
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)   # 2
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')   # 3

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,일반서비스,3860,-60,-1.53,174675649740
1,006840,AK홀딩스,KOSPI,기타금융,10430,70,0.68,138172061230
2,027410,BGF,KOSPI,기타금융,3340,-25,-0.74,319694081940
3,282330,BGF리테일,KOSPI,유통,104400,-300,-0.29,1804439786400
4,138930,BNK금융지주,KOSPI,기타금융,10540,190,1.84,3355762290260


1. OTP를 제출할 URL을 down_url에 입력한다.
2. post() 함수를 통해 위에서 부여받은 OTP 코드를 해당 URL에 제출한다.
3. 받은 데이터의 content 부분을 BytesIO()를 이용해 바이너리 스트림 형태로 만든 후, read_csv() 함수를 통해
   데이터를 읽어 온다. 해당 데이터는 EUC-KR 형태로 인코딩 되어 있으므로 이를 선언해 준다.

- 코스닥 시장의 데이터도 동일한 과정을 통해 다운로드 받는다.

In [5]:
gen_otp_ksq = {
    'mktId': 'KSQ',   # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,의료·정밀기기,1850,25,1.37,98159224000
1,054620,APS,KOSDAQ,금융,6210,120,1.97,123543112410
2,265520,AP시스템,KOSDAQ,기계·장비,17140,490,2.94,261923555940
3,211270,AP위성,KOSDAQ,운송장비·부품,12950,70,0.54,195315836800
4,109960,AP헬스케어,KOSDAQ,유통,483,-13,-2.62,96981216927


- 코스피 데이터와 코스닥 데이터를 하나로 합친다.

In [6]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)   # 1
krx_sector['종목명'] = krx_sector['종목명'].str.strip()   # 2
krx_sector['기준일'] = biz_day   # 3

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,일반서비스,3860,-60,-1.53,174675649740,20250312
1,006840,AK홀딩스,KOSPI,기타금융,10430,70,0.68,138172061230,20250312
2,027410,BGF,KOSPI,기타금융,3340,-25,-0.74,319694081940,20250312
3,282330,BGF리테일,KOSPI,유통,104400,-300,-0.29,1804439786400,20250312
4,138930,BNK금융지주,KOSPI,기타금융,10540,190,1.84,3355762290260,20250312


1. concat() 함수를 통해 두 데이터를 합쳐 주며, reset_index() 메서드를 통해 인덱스를 리셋시킨다.
   또한, drop=True를 통해 인덱스로 세팅한 열을 삭제한다.
2. 종목명에 공백이 있는 경우가 있으므로 strip() 메서드를 이용해 이를 제거해준다.
3. 데이터의 기준일에 해당하는 [기준일] 열을 추가한다.

#### 개별종목 지표 크롤링

In [7]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader',   
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,060310,3S,1850,25,1.37,54.0,34.26,NaN,NaN,998.0,1.85,0,0.00
1,095570,AJ네트웍스,3860,-60,-1.53,367.0,10.52,712.0,5.42,9326.0,0.41,270,6.99
2,006840,AK홀딩스,10430,70,0.68,2635.0,3.96,NaN,NaN,44339.0,0.24,200,1.92
3,054620,APS,6210,120,1.97,667.0,9.31,NaN,NaN,11683.0,0.53,0,0.00
4,265520,AP시스템,17140,490,2.94,3997.0,4.29,NaN,NaN,21396.0,0.80,270,1.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,000540,흥국화재,3350,105,3.24,4664.0,0.72,NaN,NaN,20881.0,0.16,0,0.00
2699,000545,흥국화재우,4935,210,4.44,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00
2700,003280,흥아해운,1719,-7,-0.41,142.0,12.11,NaN,NaN,690.0,2.49,0,0.00
2701,037440,희림,5370,-80,-1.47,489.0,10.98,NaN,NaN,5583.0,0.96,150,2.79


#### 데이터 정리하기
- 두 데이터에 공통으로 존재하지 않는 종목, 즉 하나의 데이터에만 존재하는 종목을 살펴본다.

In [8]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['상지건설 [락]', '상지건설', '신한서부티엔디리츠', '컬러레이', '코오롱티슈진', '맥쿼리인프라', 'ESR켄달스퀘어리츠', '케이탑리츠', '소마젠', 'NH프라임리츠', 'SBI핀테크솔루션즈', 'SK리츠', '헝셩그룹', '한화리츠', '한국ANKOR유전', '엑세스바이오', '미래에셋글로벌리츠', '네오이뮨텍', '롯데리츠', '이지스레지던스리츠', '신한글로벌액티브리츠', 'KB스타리츠', '신한알파리츠', '에이리츠', '크리스탈신소재', 'NH올원리츠', '미래에셋맵스리츠', '맵스리얼티1', '글로벌에스엠', 'KB발해인프라', '잉글우드랩', '마스턴프리미어리츠', '이리츠코크렙', '오가닉티코스메틱', '로스웰', 'GRT', '이스트아시아홀딩스', '코람코더원리츠', '제이알글로벌리츠', '씨엑스아이', '프레스티지바이오파마', '스타에스엠리츠', '애머릿지', '한국패러랠', '디앤디플랫폼리츠', '이지스밸류리츠', '윙입푸드', '엘브이엠씨홀딩스', '고스트스튜디오', '코람코라이프인프라리츠', 'JTC', '삼성FN리츠']


두 데이터의 종목명 열을 세트 형태로 변경한 후 symmetric_difference() 메서드를 통해 하나의 데이터에만  
있는 종목을 살펴보면 위와 같다. 해당 종목들은 선박펀드, 광물펀드, 해외종목 등 일반적이지 않은 종목들이다.

- 두 데이터를 합친다.

In [9]:
kor_ticker = pd.merge(krx_sector,
                     krx_ind,
                     on=krx_sector.columns.intersection(
                         krx_ind.columns).tolist(),
                     how='outer')

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,일반서비스,3860,-60,-1.53,1.746756e+11,20250312,367.0,10.52,712.0,5.42,9326.0,0.41,270.0,6.99
1,006840,AK홀딩스,KOSPI,기타금융,10430,70,0.68,1.381721e+11,20250312,2635.0,3.96,NaN,NaN,44339.0,0.24,200.0,1.92
2,027410,BGF,KOSPI,기타금융,3340,-25,-0.74,3.196941e+11,20250312,813.0,4.11,695.0,4.81,17286.0,0.19,120.0,3.59
3,282330,BGF리테일,KOSPI,유통,104400,-300,-0.29,1.804440e+12,20250312,11337.0,9.21,12522.0,8.34,62197.0,1.68,4100.0,3.93
4,138930,BNK금융지주,KOSPI,기타금융,10540,190,1.84,3.355762e+12,20250312,1905.0,5.53,2677.0,3.94,31746.0,0.33,510.0,4.84


merge() 함수는 on 조건을 기준으로 두 데이터를 하나로 합치며, intersectio() 메서드를 이용해 공통으로  
존재하는 [종목코드, 코드명, 종가, 대비, 등락률] 열을 기준으로 입력해 준다. 또한, 방법(how)에는 outer를 입력해준다.

- 일반적인 종목과 스팩, 우선주, 리츠, 기타 주식을 구분해 준다.

Note: 스팩(Special Purpose Acquisition Company, SPAC)이란 기업인수를 목적으로 하는 페이퍼컴퍼니를 뜻한다.  
대부분 증권사 주관으로 설립되며, 스팩이 먼저 투자자들의 자금을 모아 주식 시장에 상장이 되고 나면, 그 이후에  
괜찮은 비상장기업을 찾아 합병하는 방식으로 최종 기업 인수가 이루어진다.

In [10]:
print(kor_ticker[kor_ticker['종목명'].str.contains('스팩[제[0-9]+호')]['종목명'].values)   # 1

['DB금융스팩11호' 'DB금융스팩12호' '미래에셋드림스팩1호' '미래에셋비전스팩2호' '미래에셋비전스팩3호'
 '미래에셋비전스팩4호' '미래에셋비전스팩5호' '미래에셋비전스팩6호' '미래에셋비전스팩7호' '삼성스팩7호' '삼성스팩8호'
 '삼성스팩9호' '신영스팩10호' '신영스팩8호' '신영스팩9호' '엔에이치스팩24호' '엔에이치스팩25호' '엔에이치스팩26호'
 '엔에이치스팩27호' '엔에이치스팩29호' '엔에이치스팩30호' '엔에이치스팩31호' '유진스팩10호' '유진스팩11호'
 '유진스팩9호' '이베스트스팩6호']


In [11]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)   # 2

['BYC우' 'CJ4우(전환)' 'CJ씨푸드1우' 'CJ우' 'CJ제일제당 우' 'DL우' 'DL이앤씨2우(전환)' 'DL이앤씨우'
 'GS우' 'JW중외제약2우B' 'JW중외제약우' 'LG생활건강우' 'LG우' 'LG전자우' 'LG화학우' 'LX하우시스우'
 'LX홀딩스1우' 'NH투자증권우' 'NPC우' 'S-Oil우' 'SK디스커버리우' 'SK우' 'SK이노베이션우' 'SK증권우'
 'SK케미칼우' '계양전기우' '금강공업우' '금호건설우' '금호석유우' '깨끗한나라우' '남선알미우' '남양유업우' '넥센우'
 '넥센타이어1우B' '노루페인트우' '노루홀딩스우' '녹십자홀딩스2우' '대교우B' '대덕1우' '대덕전자1우' '대상우'
 '대상홀딩스우' '대신증권2우B' '대신증권우' '대원전선우' '대한제당우' '대한항공우' '덕성우' '동부건설우' '동양2우B'
 '동양우' '동원시스템즈우' '두산2우B' '두산우' '두산퓨얼셀1우' '두산퓨얼셀2우B' '롯데지주우' '롯데칠성우'
 '미래에셋증권2우B' '미래에셋증권우' '부국증권우' '삼성SDI우' '삼성물산우B' '삼성전기우' '삼성전자우' '삼성화재우'
 '삼양사우' '삼양홀딩스우' '서울식품우' '성문전자우' '성신양회우' '세방우' '솔루스첨단소재1우' '솔루스첨단소재2우B'
 '신풍제약우' '아모레G3우(전환)' '아모레G우' '아모레퍼시픽우' '유안타증권우' '유유제약1우' '유유제약2우B'
 '유한양행우' '유화증권우' '일양약품우' '진흥기업2우B' '진흥기업우B' '코리아써우' '코리아써키트2우B' '코오롱글로벌우'
 '코오롱모빌리티그룹우' '코오롱우' '코오롱인더우' '크라운제과우' '크라운해태홀딩스우' '태양금속우' '태영건설우'
 '티와이홀딩스우' '하이트진로2우B' '하이트진로홀딩스우' '한국금융지주우' '한양증권우' '한진칼우' '한화3우B'
 '한화갤러리아우' '한화솔루션우' '한화우' '한화투자증권우' '현대건설우' '현대차2우B' '현대차3우B' '현대차우'
 '호텔신라

In [12]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)   # 3

['ESR켄달스퀘어리츠' 'KB스타리츠' 'NH올원리츠' 'NH프라임리츠' 'SK리츠' '디앤디플랫폼리츠' '롯데리츠'
 '마스턴프리미어리츠' '미래에셋글로벌리츠' '미래에셋맵스리츠' '삼성FN리츠' '스타에스엠리츠' '신한글로벌액티브리츠'
 '신한서부티엔디리츠' '신한알파리츠' '에이리츠' '이지스레지던스리츠' '이지스밸류리츠' '제이알글로벌리츠' '케이탑리츠'
 '코람코더원리츠' '코람코라이프인프라리츠' '한화리츠']


1. 스팩 종목은 종목명에 '스팩' 또는 '제n호'라는 단어가 들어간다. 따라서 contains() 메서드를 통해 종목명에
   '스팩'이 들어가거나 정규 표현식을 이요해 '제n호'라는 문자가 들어간 종목명을 찾는다.
2. 국내 종목 중 종목코드 끝이 0이 아닌 종목은 우선주에 해당한다.
3. 리츠 종목은 종목명이 '리츠'로 끝난다. 따라서 endswith() 메서드를 통해 이러한 종목을 찾는다.
   (메리츠화재 등의 종목도 중간에 리츠라는 단어가 들어가므로 contains() 함수를 이용하면 안 된다.)

- 해당 종목들을 구분하여 표기해 준다.

In [13]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                             np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                      np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                              np.where(kor_ticker['종목명'].isin(diff), '기타', 
                                                       '보통주'))))   # 1

kor_ticker = kor_ticker.reset_index(drop=True)   # 2
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')   # 3
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가', '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]   # 4
kor_ticker = kor_ticker.replace({np.nan: None})   # 5
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])   # 6

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,3860,174675649740.0,2025-03-12,367.0,712.0,9326.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,10430,138172061230.0,2025-03-12,2635.0,None,44339.0,200.0,보통주
2,027410,BGF,KOSPI,3340,319694081940.0,2025-03-12,813.0,695.0,17286.0,120.0,보통주
3,282330,BGF리테일,KOSPI,104400,1804439786400.0,2025-03-12,11337.0,12522.0,62197.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,10540,3355762290260.0,2025-03-12,1905.0,2677.0,31746.0,510.0,보통주


1. numpy 패키지의 where() 함수를 통해 각 조건에 맞는 종목구분을 입력한다.
   종목명에 '스팩' 또는 '제n호'가 포함된 종목은 스팩으로, 종목코드 끝이 0이 아닌 종목은 '우선주'로,
   종목명이 '리츠'로 끝나는 종목은 '리츠'로, 선박펀드, 광물펀드, 해외종목 등은 '기타'로,  
   나머지 종목들은 '보통주'로 구분한다.
2. reset_index() 메서드를 통해 인덱스를 초기화한다.
3. replace() 메서드를 통해 열 이름의 공백을 삭제한다.
4. 필요한 열만 선택한다.
5. SQL에는 NaN이 입력되지 않으므로, None으로 변경한다.
6. 기준일을 to_datetime() 메서드를 이용해 yyyymmdd에서 yyyy-mm-dd 형태로 변경한다.

- 해당 정보를 DB에 저장한다.

MySQL에서 아래의 쿼리를 입력해 데이터베이스(stock_db)를 만든 후, 국내 티커 정보가 들어갈 테이블(kor_ticker)을 만들어 준다.
```SQL
create database stock_db;

use stock_db;

create table kor_ticker (
    종목코드 varchar(6) not null,
    종목명 varchar(20),
    시장구분 varchar(6),
    종가 float,
    시가총액 float,
    기준일 date,
    EPS float,
    선행EPS float,
    BPS float,
    주당배당금 float,
    종목구분 varchar(5),
    primary key(종목코드, 기준일)
);
```

- primary key인 '기준일' 열에 NaN 값이 존재하면 안되므로 해당 행 삭제

In [14]:
kor_ticker = kor_ticker.dropna(subset=['기준일'])

- kor_ticker 테이블에 upsert 형태로 저장한다.

In [15]:
import pymysql

con = pymysql.connect(user='root',
                     passwd='1234',
                     host='127.0.0.1',
                     db='stock_db',
                     charset='utf8')

mycursor = con.cursor()
query = f"""
    insert into kor_ticker (종목코드, 종목명, 시장구분, 종가, 시가총액, 기준일, EPS, 선행EPS, BPS, 주당배당금, 종목구분)
    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) as new
    on duplicate key update
    종목명=new.종목명, 시장구분=new.시장구분, 종가=new.종가, 시가총액=new.시가총액, EPS=new.EPS, 선행EPS=new.선행EPS, BPS=new.BPS, 주당배당금=new.주당배당금, 종목구분=new.종목구분;
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()